In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

get_ipython().run_line_magic('matplotlib', 'inline')
pd.set_option('display.max_rows', 500)

In [2]:
pd_JH_data=pd.read_csv('C:/Users/asus/ads_covid-19/data/processed/COVID_relational_confirmed.csv',sep=':',parse_dates=[0])
pd_JH_data

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,Anguilla,United Kingdom,0
2,2020-01-22,Anhui,China,1
3,2020-01-22,Aruba,Netherlands,0
4,2020-01-22,Australian Capital Territory,Australia,0
...,...,...,...,...
52131,2020-08-04,no,West Bank and Gaza,12770
52132,2020-08-04,no,Western Sahara,10
52133,2020-08-04,no,Yemen,1760
52134,2020-08-04,no,Zambia,6793


In [3]:
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()


,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,"Korea, South",1
2,2020-01-22,no,Kosovo,0
3,2020-01-22,no,Kuwait,0
4,2020-01-22,no,Kyrgyzstan,0


In [4]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]


In [5]:
test_data.head()

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213
15791,2020-03-21,no,US,25825
16002,2020-03-22,no,Germany,24873
16043,2020-03-22,no,US,33761
16268,2020-03-23,no,Germany,29056


In [6]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-08-04,no,212828
US,2020-08-04,no,4771080


In [7]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)


In [8]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [9]:
test_data.groupby(['state','country']).agg(np.max)



date  confirmed
state country                      
no    Germany 2020-08-04     212828
      US      2020-08-04    4771080

In [10]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [11]:
test_data

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213
15791,2020-03-21,no,US,25825
16002,2020-03-22,no,Germany,24873
16043,2020-03-22,no,US,33761
16268,2020-03-23,no,Germany,29056
16308,2020-03-23,no,US,43850
16535,2020-03-24,no,Germany,32986
16575,2020-03-24,no,US,54112
16800,2020-03-25,no,Germany,37323
16840,2020-03-25,no,US,66055


In [12]:

test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')


state  country       
no     Germany  15736            NaN
                16002            NaN
                16268       7.417994
                16535       7.142035
                16800       8.012983
                17066       6.954407
                17332       6.501919
                17598       7.390371
                17863      10.136671
                18131      13.541893
                18396      13.781393
                18662      13.140681
                18929      12.037271
                19195      12.735506
                19460      16.052694
                19727      21.372453
                19992      27.427355
                20257      27.511936
                20525      21.792179
                20790      21.495848
                21055      26.565108
                21323      36.198404
                21588      43.982990
                21854      49.423444
                22121      74.043747
                22387      56.424411
                

In [13]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [14]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'doubling_rate',
                             'level_2':'index'})
pd_DR_result.head()


,state,country,index,doubling_rate
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1197,NaN


In [15]:

pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()


,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,"Korea, South",1
2,2,2020-01-22,no,Kosovo,0
3,3,2020-01-22,no,Kuwait,0
4,4,2020-01-22,no,Kyrgyzstan,0


In [16]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','doubling_rate']],on=['index'],how='left')
pd_result_larg.head()


,index,date,state,country,confirmed,doubling_rate
0,0,2020-01-22,Alberta,Canada,0,NaN
1,1,2020-01-22,no,"Korea, South",1,NaN
2,2,2020-01-22,no,Kosovo,0,NaN
3,3,2020-01-22,no,Kuwait,0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0,NaN


In [17]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [18]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

pd_filtered_result

,index,state,country,confirmed,confirmed_filtered
0,0,Alberta,Canada,0,0.0
1,1,no,"Korea, South",1,0.8
2,2,no,Kosovo,0,0.0
3,3,no,Kuwait,0,0.0
4,4,no,Kyrgyzstan,0,0.0
...,...,...,...,...,...
52131,52131,no,Barbados,132,136.4
52132,52132,no,Belarus,68250,68268.2
52133,52133,no,Belgium,70648,70734.0
52134,52134,no,Albania,5750,5746.6


In [19]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1,NaN,0.8
2,2,2020-01-22,no,Kosovo,0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0,NaN,0.0


In [20]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'doubling_rate_filtered',
                             'level_2':'index'})

pd_filtered_doubling.head()

,state,country,index,doubling_rate_filtered
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1197,NaN


In [21]:

pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','doubling_rate_filtered']],on=['index'],how='left')
pd_result_larg.head()


,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,doubling_rate_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1,NaN,0.8,NaN
2,2,2020-01-22,no,Kosovo,0,NaN,0.0,NaN
3,3,2020-01-22,no,Kuwait,0,NaN,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0,NaN,0.0,NaN


In [22]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['doubling_rate_filtered']=pd_result_larg['doubling_rate_filtered'].where(mask, other=np.NaN) 



In [23]:
pd_result_larg.head()


,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,doubling_rate_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1,NaN,0.8,NaN
2,2,2020-01-22,no,Kosovo,0,NaN,0.0,NaN
3,3,2020-01-22,no,Kuwait,0,NaN,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0,NaN,0.0,NaN


In [24]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)